# Localized Homicide Clearance Rates, 1995-present


In [1]:
import os
import sqlite3
import pandas as pd


## Data Import

In [2]:
def should_chdir():
    """checks whether the code is being run interactively or from command line"""
    # if both these directories exist, treat is as the correct directory
    dirs = ["input", "output"]
    if all(os.path.exists(d) for d in dirs):
        return False

    try:
        shell = get_ipython().__class__.__name__  # type: ignore
        if shell == "ZMQInteractiveShell":
            return True
        elif shell == "TerminalInteractiveShell":
            return False
        else:
            return False
    except NameError:
        return False


# step backwards if notebook is being run interactively
# or stay put if being run from command line
if should_chdir():
    os.chdir("..")


In [3]:
with sqlite3.connect("input/sqlite__temp.db") as con:
    agencies = pd.read_sql("SELECT * FROM data_agencies;", con)
    gb_cols = "ori_code, card, year"
    reta = pd.read_sql(
        f"""
        SELECT {gb_cols}, SUM(value) as total
        FROM data_reta
        GROUP BY {gb_cols};
    """,
        con,
    )


### Merging geographies

In [4]:
df = reta.merge(
    agencies,
    how="left",
    left_on=["ori_code", "year"],
    right_on=["ori", "data_year"],
)


### Filtering unwanted records

In [5]:
# FDLK24000 is for federal agencies, which all use the same ORI and won't be included here
df = df[df.ori_code != "FDLK24000"]


#### Selecting data only from agencies that regularly reported >1 clearance

This data contains zeros for both agencies that reported no data, and agencies that actually reported zero clearances. For example, the Chicago Police Department reported hundreds of homicides each year but no clearances at all

In [6]:
df[df.ori_code == "ILCPD0000"].pivot(index="year", columns="card", values="total")


card,actual,cleared_arrest
year,,
1995,824,0
1996,789,0
1997,757,0
1998,703,0
1999,642,0
2000,631,0
2001,666,0
2002,648,0
2003,598,0


To address this, I'm only going to look at agencies that reported at least 1 clearance to the FBI in at least ~90% of the years we're looking at; in this case, 23 out of the 26 years (~88.5%). This approach is adapted from [Jeremy Singer-Vine's excellent analysis](https://github.com/the-trace-and-buzzfeed-news/federal-crime-data-analysis/blob/master/notebooks/analyze/00-analyze-reta.ipynb) of this data in 2018 for Buzzfeed News. 

In [7]:
MIN_NONZERO_CLEARANCE_YEARS = 23
INCLUDE_AGENCIES_MERGE_COLS = ["ori_code"]

include_agencies = (
    pd.crosstab(
        index=[df.year, df.ori_code],
        columns=df.card,
        values=df.total,
        aggfunc=sum,
    )
    .assign(nonzero_clearance=lambda df: df.cleared_arrest > 0)
    .reset_index()
    .groupby(INCLUDE_AGENCIES_MERGE_COLS)
    .nonzero_clearance.sum()
    .to_frame("nonzero_clearance_years")
    .query("nonzero_clearance_years > @MIN_NONZERO_CLEARANCE_YEARS")
)

include_agencies


,nonzero_clearance_years
ori_code,
AK0010100,26
AL0020000,24
AL0110100,26
AL0630000,24
AR0160100,25
...,...
WI0130100,24
WI0520200,24
WIMPD0000,26


Select only rows in the original dataframe that are in include_agencies

In [8]:
df = df.merge(
    include_agencies, how="right", left_on=INCLUDE_AGENCIES_MERGE_COLS, right_index=True
).drop("nonzero_clearance_years", axis=1)

df


,ori_code,card,year,total,unique_id,data_year,ori,ncic_agency_name,state_abbr,county_name,msa_name,city_name
0,AK0010100,actual,1995,29,00be478396bcfdd84f9361c6ed83ca28b167b0fd,1995.0,AK0010100,ANCHORAGE PD,AK,ANCHORAGE,"Ames, IA",ANCHORAGE
1,AK0010100,actual,1996,25,ccc6735b0cb84b86330d20e09a4105a29542a540,1996.0,AK0010100,ANCHORAGE PD,AK,ANCHORAGE,"Ames, IA",ANCHORAGE
2,AK0010100,actual,1997,23,ec895861f02e61281526c06303cffaf7f160c2df,1997.0,AK0010100,ANCHORAGE PD,AK,ANCHORAGE,"Ames, IA",ANCHORAGE
3,AK0010100,actual,1998,19,16c8416a45081736b85ca65ab08f492bf65d8e33,1998.0,AK0010100,ANCHORAGE PD,AK,ANCHORAGE,"Ames, IA",ANCHORAGE
4,AK0010100,actual,1999,19,4503dee76665fda72813b58373430cf6f3eca029,1999.0,AK0010100,ANCHORAGE PD,AK,ANCHORAGE,"Ames, IA",ANCHORAGE
...,...,...,...,...,...,...,...,...,...,...,...,...
1083121,WV0200200,cleared_arrest,2016,5,2ea033e3f0fe5025cc2d32cf1b8c7f101da3a859,2016.0,WV0200200,CHARLESTON PD,WV,KANAWHA,"Charleston, WV",CHARLESTON
1083122,WV0200200,cleared_arrest,2017,6,442ed5a00abbbaefa2eb14c8117ac0b0b9cf5e7f,2017.0,WV0200200,CHARLESTON PD,WV,KANAWHA,"Charleston, WV",CHARLESTON
1083123,WV0200200,cleared_arrest,2018,3,2c33fb670bcb80354d495a93da69f0825ba01e8a,2018.0,WV0200200,CHARLESTON PD,WV,KANAWHA,"Charleston, WV",CHARLESTON
1083124,WV0200200,cleared_arrest,2019,1,574cc587168ac2792f5ca6dba04aede3c69ec7d6,2019.0,WV0200200,CHARLESTON PD,WV,KANAWHA,"Charleston, WV",CHARLESTON


## Analysis

In [9]:
# helper functions that will generate tables used in multiple tasks below


def get_clearance_rate_df(index):
    """creates a crosstab of totals and clearance rates with a given index"""

    if isinstance(index, (str, int, float)):
        index = df[index]
    elif isinstance(index, list):
        index = [df[i] for i in index]
    else:
        raise ValueError("index must be a column name or list of column names")

    return pd.crosstab(
        index=index, columns=df.card, values=df.total, aggfunc=sum
    ).assign(clearance_rate=lambda d: d.cleared_arrest / d.actual)


def get_max_year(clearance_df):
    return clearance_df[
        (clearance_df.actual.notna())
        & (clearance_df.cleared_arrest.notna())
        & (clearance_df.actual > 0)
        & (clearance_df.cleared_arrest > 0)
    ].year.max()


def get_latest(clearance_df, groupby_col):
    """generates a subset of clearance_df with only numbers for the latest complete year"""
    dfs = []
    for gbval in clearance_df[groupby_col].unique():
        subdf = clearance_df[clearance_df[groupby_col] == gbval].copy()
        subdf = subdf[subdf.year == get_max_year(subdf)]
        dfs.append(subdf)
    clearance_df_latest = pd.concat(dfs).rename(columns={"year": "latest_year"})
    return clearance_df_latest


def get_5yravg_vs_latest(clearance_df, groupby_col, remerge_col=None):
    """
    generates a dataframe with the 2020 clearance rate, the 2015-2019 average,
    and a percent change column
    """
    avgcol = "5_year_avg"
    latestcol = "latest"

    groupby_cols = [groupby_col]
    if remerge_col is not None:
        groupby_cols.append(remerge_col)

    dfs = []
    for gbval in clearance_df[groupby_col].unique():
        subdf = clearance_df[clearance_df[groupby_col] == gbval].copy()
        max_year = get_max_year(subdf)
        avg_start = max_year - 5
        dfs.append(
            subdf[subdf.year.isin(range(avg_start, max_year))]
            .groupby(groupby_cols)
            .clearance_rate.mean()
            .to_frame(avgcol)
            .join(
                subdf[subdf.year == max_year].set_index(groupby_cols)[
                    ["clearance_rate"]
                ].rename(columns={"clearance_rate": latestcol})
            )
            .assign(change=lambda df: (df[latestcol] - df[avgcol]) / df[avgcol])
            .assign(latest_year=max_year)
        )
    return pd.concat(dfs)


### National

Of these agencies we're looking at, overall, the clearance rate fell from a high of about 65% in the late 90s to about 52% in 2020.

In [10]:
national = get_clearance_rate_df("year")
national


card,actual,cleared_arrest,clearance_rate
year,,,
1995,11999,7255,0.604634
1996,10619,6419,0.604483
1997,9920,6155,0.620464
1998,8960,5773,0.644308
1999,8100,5255,0.648765
2000,8468,4955,0.585144
2001,8923,5122,0.574022
2002,9274,5541,0.597477
2003,9392,5574,0.593484


In [11]:
national.to_csv("output/national.csv")


### By state

In [12]:
state = get_clearance_rate_df(["year", "state_abbr"]).reset_index()
state.columns.name = None
state


,year,state_abbr,actual,cleared_arrest,clearance_rate
0,1995,AK,29,19,0.655172
1,1995,AL,11,6,0.545455
2,1995,AR,115,75,0.652174
3,1995,AZ,387,216,0.558140
4,1995,CA,2585,1406,0.543907
...,...,...,...,...,...
983,2020,UT,23,13,0.565217
984,2020,VA,337,200,0.593472
985,2020,WA,147,80,0.544218
986,2020,WI,209,120,0.574163


In [13]:
# output for use in later tasks
state.to_csv("output/state.csv", index=False)


Clearance rates can vary wildly from state to state. Some are as low as 11%, while others are more than 70%

In [14]:
state_latest = get_latest(state, "state_abbr")

state_latest.sort_values("clearance_rate")


,latest_year,state_abbr,actual,cleared_arrest,clearance_rate
979,2020,RI,17,2,0.117647
973,2020,NM,89,20,0.224719
958,2020,DE,46,12,0.260870
960,2020,GA,231,71,0.307359
965,2020,MA,90,30,0.333333
962,2020,IN,293,106,0.361775
987,2020,WV,24,9,0.375000
976,2020,OH,551,208,0.377495
978,2020,PA,248,96,0.387097
975,2020,NY,184,72,0.391304


In [15]:
# output for later use
state_latest.to_csv(f"output/state_latest.csv", index=False)


Most states saw a decline in clearance rates in 2020 compared to the previous 5 years


In [16]:
state_5yr = get_5yravg_vs_latest(state, "state_abbr")

state_5yr.sort_values("change")


,5_year_avg,latest,change,latest_year
state_abbr,,,,
NM,0.481300,0.224719,-0.533100,2020
DE,0.528328,0.260870,-0.506236,2020
RI,0.235531,0.117647,-0.500503,2020
GA,0.574649,0.307359,-0.465135,2020
TN,0.560806,0.396552,-0.292889,2020
PA,0.536473,0.387097,-0.278441,2020
NY,0.541996,0.391304,-0.278031,2020
UT,0.753808,0.565217,-0.250184,2020
MA,0.441995,0.333333,-0.245844,2020


In [17]:
# output for later tasks
state_5yr.to_csv("output/state_5yr.csv")


### Localized analysis

The cells below are used to calculate columns that will be used in later analyses

#### By MSA

In [18]:
msa = get_clearance_rate_df(["year", "msa_name"]).reset_index()

msa


card,year,msa_name,actual,cleared_arrest,clearance_rate
0,1995,"Abilene, TX",5,2,0.400000
1,1995,"Albany, GA",14,14,1.000000
2,1995,"Albany-Schenectady-Troy, NY",15,10,0.666667
3,1995,"Albuquerque, NM",53,9,0.169811
4,1995,"Alexandria, LA",7,7,1.000000
...,...,...,...,...,...
4684,2020,"Wichita Falls, TX",9,6,0.666667
4685,2020,"Wilmington, NC",22,14,0.636364
4686,2020,"Winston-Salem, NC",2,2,1.000000
4687,2020,"Worcester, MA-CT",10,6,0.600000


In [19]:
msa.to_csv("output/msa.csv", index=False)


In [20]:
msa_latest = get_latest(msa, "msa_name")

# looking here only at agencies with 50 or more homicides in 2020
msa_latest.query("actual > 50").dropna(subset=["clearance_rate"])


card,latest_year,msa_name,actual,cleared_arrest,clearance_rate
4503,2020,"Albuquerque, NM",80,14,0.175000
4510,2020,"Atlanta-Sandy Springs-Alpharetta, GA",205,55,0.268293
4514,2020,"Bakersfield, CA",103,34,0.330097
4515,2020,"Baltimore-Columbia-Towson, MD",345,169,0.489855
4516,2020,"Baton Rouge, LA",114,72,0.631579
4519,2020,"Boston-Cambridge-Newton, MA-NH",62,19,0.306452
4523,2020,"Buffalo-Cheektowaga, NY",61,15,0.245902
4527,2020,"Charleston-North Charleston, SC",76,45,0.592105
4528,2020,"Charlotte-Concord-Gastonia, NC-SC",144,97,0.673611
4530,2020,"Cincinnati, OH-KY-IN",94,41,0.436170


In [21]:
# output for use in later tasks
msa_latest.to_csv("output/msa_latest.csv", index=False)


In [22]:
msa_5yr = get_5yravg_vs_latest(msa, "msa_name")

msa_5yr.dropna(subset=["change"]).sort_values("change")


,5_year_avg,latest,change,latest_year
msa_name,,,,
"Flint, MI",0.108120,0.022727,-0.789796,2020
"Vallejo, CA",0.701190,0.151515,-0.783917,2020
"Visalia, CA",1.293121,0.294118,-0.772552,2020
"Raleigh-Cary, NC",0.800000,0.250000,-0.687500,2020
"Hickory-Lenoir-Morganton, NC",0.972381,0.333333,-0.657199,2020
...,...,...,...,...
"Phoenix-Mesa-Chandler, AZ; Prescott Valley-Prescott, AZ",0.766667,1.500000,0.956522,2016
"Racine, WI",0.750000,1.500000,1.000000,2020
"Lincoln, NE",0.858333,1.800000,1.097087,2020


In [23]:
# output for use in later tasks
msa_5yr.to_csv("output/msa_5yr.csv")


#### By agency

In [24]:
agency = get_clearance_rate_df(["year", "ori_code", "ncic_agency_name"]).reset_index()

agency


card,year,ori_code,ncic_agency_name,actual,cleared_arrest,clearance_rate
0,1995,AK0010100,ANCHORAGE PD,29,19,0.655172
1,1995,AL0020000,MOBILE CO SO MOBILE,3,0,0.000000
2,1995,AL0110100,ANNISTON PD,6,4,0.666667
3,1995,AL0630000,TUSCALOOSA CO SO TUSCALOOSA,2,2,1.000000
4,1995,AR0160100,JONESBORO PD,3,1,0.333333
...,...,...,...,...,...,...
9927,2020,WI0130100,MADISON PD,10,7,0.700000
9928,2020,WI0520200,RACINE PD,2,3,1.500000
9929,2020,WIMPD0000,MILWAUKEE PD,191,103,0.539267
9930,2020,WV0200000,KANAWHA CO SO CHARLESTON,13,6,0.461538


In [25]:
agency.to_csv("output/agency.csv", index=False)


In [26]:
agency_latest = get_latest(agency, "ori_code")

# using this in later step
agency_latest_subset = agency_latest.query("actual > 25").dropna(subset=["clearance_rate"])

# again looking only at ones with many homicides in latest
agency_latest_subset.sort_values("clearance_rate")


card,latest_year,ori_code,ncic_agency_name,actual,cleared_arrest,clearance_rate
9727,2020,MI2539800,FLINT PD,44,1,0.022727
9632,2020,CA0480700,VALLEJO PD,27,1,0.037037
9808,2020,OHCLP0000,CLEVELAND,160,17,0.106250
9785,2020,NM0010100,ALBUQUERQUE PD,80,14,0.175000
9585,2020,CA0150000,KERN CO SO BAKERSFIELD,59,11,0.186441
...,...,...,...,...,...,...
9890,2020,TX2270100,AUSTIN PD,44,38,0.863636
9743,2020,NB0280200,OMAHA PD,37,32,0.864865
9672,2020,FL0480400,ORLANDO PD,31,27,0.870968
9699,2020,KY0340200,LEXINGTON-FAYETTE URBAN COUNTY DIV OF POLICE,28,26,0.928571


In [27]:
# output for use in later tasks
agency_latest.to_csv("output/agency_latest.csv", index=False)


In [29]:
agency_5yr = get_5yravg_vs_latest(agency, "ori_code", "ncic_agency_name")

agency_5yr[
    agency_5yr.index.get_level_values("ori_code").isin(
        agency_latest_subset.ori_code
    )
].sort_values("change")


,,5_year_avg,latest,change,latest_year
ori_code,ncic_agency_name,,,,
CA0480700,VALLEJO PD,0.620556,0.037037,-0.940316,2020
MI2539800,FLINT PD,0.108120,0.022727,-0.789796,2020
GAAPD0000,ATLANTA POLICE DEPARTMENT,0.804225,0.237288,-0.704948,2020
NJ0160800,PATERSON PD,0.687893,0.222222,-0.676952,2020
NM0010100,ALBUQUERQUE PD,0.481177,0.175000,-0.636308,2020
...,...,...,...,...,...
FL0480400,ORLANDO PD,0.576945,0.870968,0.509619,2020
GA0440200,DE KALB COUNTY PD DECATUR,0.122599,0.214286,0.747857,2020
CA0390500,STOCKTON PD,0.368760,0.660714,0.791720,2020


In [30]:
agency_5yr.to_csv("output/agency_5yr.csv")
